In [1]:
#dependencies

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
#read csv, push to dataframe

us_df = pd.read_csv ("data/state_daily_cases.csv")
us_df.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200825,AK,5666.0,328165.0,NaN,46.0,NaN,NaN,NaN,6.0,...,333831,4,0,bd407e9ed01cce1ea303fe93d06cd759e6488332,0,0,0,0,0,NaN
1,20200825,AL,117242.0,810035.0,NaN,1097.0,13793.0,NaN,1399.0,NaN,...,927277,13,0,4945c5836682d749c8ab2609d9b6dce0b72b60ce,0,0,0,0,0,NaN
2,20200825,AR,57374.0,616810.0,NaN,442.0,4013.0,NaN,NaN,108.0,...,674184,15,51,803f6524a9f20c7d7f691136319ff875adac2d76,0,0,0,0,0,NaN
3,20200825,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e762458f6ad93db951259394cf32880309d5dfc4,0,0,0,0,0,NaN
4,20200825,AZ,199273.0,962931.0,NaN,999.0,21365.0,319.0,NaN,168.0,...,1162204,21,146,df43531eadb8a5893d2c552b4285f1b00e483345,0,0,0,0,0,NaN


In [3]:
#isolate data on georgia, remove extraneous columns, reset index

ga_df = us_df.loc [us_df['state'] == 'GA', ['date', 'state', 'positive', 'hospitalizedCurrently', 'hospitalizedCumulative', 'inIcuCumulative']]
ga_df = ga_df.reset_index (drop = True)

ga_df

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCumulative
0,20200825,GA,258354.0,2260.0,23717.0,4322.0
1,20200824,GA,256253.0,2350.0,23425.0,4272.0
2,20200823,GA,253949.0,2360.0,23369.0,4265.0
3,20200822,GA,252222.0,2361.0,23325.0,4251.0
4,20200821,GA,249630.0,2408.0,23125.0,4218.0
...,...,...,...,...,...,...
170,20200308,GA,7.0,NaN,NaN,NaN
171,20200307,GA,6.0,NaN,NaN,NaN
172,20200306,GA,2.0,NaN,NaN,NaN
173,20200305,GA,2.0,NaN,NaN,NaN


In [12]:
#loop through rows, calculate days from start of pandemic in GA, insert into new column

ga_df['days_from_start'] = ''
ga_df['date_format'] = ''

start_day = dt.datetime.strptime (f'{(ga_df.iloc [-1, 0])}', '%Y%m%d')

for index, row in ga_df.iterrows():
    day_number = (dt.datetime.strptime (f"{row['date']}", '%Y%m%d'))
    day_delta = (day_number - start_day).days
    ga_df.loc [index, 'date_format'] = day_number
    ga_df.loc [index, 'days_from_start'] = day_delta
    
ga_df

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCumulative,days_from_start,date_format,daily_increase
0,20200825,GA,258354.0,2260.0,23717.0,4322.0,174,2020-08-25 00:00:00,2101
1,20200824,GA,256253.0,2350.0,23425.0,4272.0,173,2020-08-24 00:00:00,2304
2,20200823,GA,253949.0,2360.0,23369.0,4265.0,172,2020-08-23 00:00:00,1727
3,20200822,GA,252222.0,2361.0,23325.0,4251.0,171,2020-08-22 00:00:00,2592
4,20200821,GA,249630.0,2408.0,23125.0,4218.0,170,2020-08-21 00:00:00,2889
...,...,...,...,...,...,...,...,...,...
170,20200308,GA,7.0,NaN,NaN,NaN,4,2020-03-08 00:00:00,1
171,20200307,GA,6.0,NaN,NaN,NaN,3,2020-03-07 00:00:00,4
172,20200306,GA,2.0,NaN,NaN,NaN,2,2020-03-06 00:00:00,0
173,20200305,GA,2.0,NaN,NaN,NaN,1,2020-03-05 00:00:00,0


In [77]:
#loop through cases column, calculate daily increase, insert into new column

ga_df['daily_increase'] = ''

for x in range(len(ga_df['positive'])):
    try:
        increase = ga_df['positive'][x] - ga_df['positive'][x + 1]
    
    except:
        increase = ga_df['positive'][x] - 0
        
    ga_df.iloc [x, 8] = increase
    
ga_df

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCumulative,days_from_start,date_format,daily_increase,7d_avg_increase
0,20200825,GA,258354.0,2260.0,23717.0,4322.0,174,2020-08-25 00:00:00,2101,2382.43
1,20200824,GA,256253.0,2350.0,23425.0,4272.0,173,2020-08-24 00:00:00,2304,2484.57
2,20200823,GA,253949.0,2360.0,23369.0,4265.0,172,2020-08-23 00:00:00,1727,2417
3,20200822,GA,252222.0,2361.0,23325.0,4251.0,171,2020-08-22 00:00:00,2592,2436.29
4,20200821,GA,249630.0,2408.0,23125.0,4218.0,170,2020-08-21 00:00:00,2889,2533.57
...,...,...,...,...,...,...,...,...,...,...
170,20200308,GA,7.0,NaN,NaN,NaN,4,2020-03-08 00:00:00,1,
171,20200307,GA,6.0,NaN,NaN,NaN,3,2020-03-07 00:00:00,4,
172,20200306,GA,2.0,NaN,NaN,NaN,2,2020-03-06 00:00:00,0,
173,20200305,GA,2.0,NaN,NaN,NaN,1,2020-03-05 00:00:00,0,


In [78]:
#loop through cases column, calculate 7-day average increase, insert into new column

ga_df['7d_avg_increase'] = ''

for x in range(len(ga_df['daily_increase'])):
    try:
        running_avg = (ga_df.iloc [x:(x + 7), 8]).mean()
        ga_df.iloc [x, 9] = running_avg
        
    except:
        try:
            running_avg = (ga_df.iloc [-x:, 8]).mean()
            ga_df.iloc [x, 9] = running_avg
            
        except:
            pass
    
ga_df

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCumulative,days_from_start,date_format,daily_increase,7d_avg_increase
0,20200825,GA,258354.0,2260.0,23717.0,4322.0,174,2020-08-25 00:00:00,2101,2382.43
1,20200824,GA,256253.0,2350.0,23425.0,4272.0,173,2020-08-24 00:00:00,2304,2484.57
2,20200823,GA,253949.0,2360.0,23369.0,4265.0,172,2020-08-23 00:00:00,1727,2417
3,20200822,GA,252222.0,2361.0,23325.0,4251.0,171,2020-08-22 00:00:00,2592,2436.29
4,20200821,GA,249630.0,2408.0,23125.0,4218.0,170,2020-08-21 00:00:00,2889,2533.57
...,...,...,...,...,...,...,...,...,...,...
170,20200308,GA,7.0,NaN,NaN,NaN,4,2020-03-08 00:00:00,1,1.4
171,20200307,GA,6.0,NaN,NaN,NaN,3,2020-03-07 00:00:00,4,1.5
172,20200306,GA,2.0,NaN,NaN,NaN,2,2020-03-06 00:00:00,0,0.666667
173,20200305,GA,2.0,NaN,NaN,NaN,1,2020-03-05 00:00:00,0,1


In [148]:
fig = go.Figure()

fig.add_trace (
    go.Bar (x = ga_df['date_format'],
            y = ga_df['daily_increase'],
            name = 'daily increase',
            marker = dict (color = ga_df['daily_increase'],
                           colorscale = 'BlueRed'))
)

fig.add_trace (
    go.Scatter (x = ga_df['date_format'],
                y = ga_df['7d_avg_increase'],
                name = '7-day running average',
                line = dict (color = '#FFEE58'),
                opacity = 0.5,
                fill = 'tozeroy')
)

fig.add_shape (dict (type = 'line',
                     x0 = '2020-07-18',
                     x1 = '2020-07-18',
                     y0 = 0,
                     y1 = 5000,
                     line = dict (color = 'black',
                                  width = 2,
                                  dash = 'dashdot')
                    ))

fig.add_shape (dict (type = 'line',
                     x0 = '2020-05-08',
                     x1 = '2020-05-08',
                     y0 = 0,
                     y1 = 5000,
                     line = dict (color = 'black',
                                  width = 2,
                                  dash = 'dashdot')
                    ))

fig.show()